<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/ml_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras_self_attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for keras_self_attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18913 sha256=e6bb95a462ad1a8cb3f0a50f0f754c43541eb88331161bd37c475c002461070c
  Stored in directory: /root/.cache/pip/wheels/ac/13/2d/3de7c76f618a8d162884ac5b726a8c2242ad88afa370f1e62f
Successfully built keras_self_attention


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras.layers import Dense,Flatten,Conv1D,MaxPooling1D,Dropout,BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras import Model
from keras import regularizers
from keras import initializers
from keras_self_attention import SeqSelfAttention, SeqWeightedAttention
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
import sklearn

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
dataset_path = '/content/drive/MyDrive/delesmell/METHOD_01.csv'
df = pd.read_csv(dataset_path)
df.head()

,NAME,ALD,ATFD,CALIN,CC,CCL,CDISP,CEXT,CINT,CM,...,LOC,MAXNESTING,NOAV,NOEU,NOLV,NOP,word2vec,word2vec2,word2vec3,is_brain
0,tagDefineTextField,1,0,0,0,2,0.00,0,0,0,...,5,1,2,0,0,14,0.766091,0.295311,-0.482078,False
1,tagDefineFont2,1,0,1,0,2,0.00,0,0,0,...,6,1,3,0,0,13,0.350905,0.692511,0.782078,False
2,classifyStream,0,0,32,0,3,0.00,0,0,0,...,118,3,34,0,27,11,-0.064280,1.089711,2.046234,False
3,classifyStream,0,0,17,0,3,0.17,1,6,0,...,76,3,16,0,13,10,-0.479466,1.486911,3.310390,False
4,generate,0,0,22,0,3,0.00,0,0,0,...,131,3,37,0,20,9,-0.894651,1.884111,4.574546,False


In [25]:
df.drop('NAME', inplace=True, axis=1)
df.head()

,ALD,ATFD,CALIN,CC,CCL,CDISP,CEXT,CINT,CM,CYCLO,...,LOC,MAXNESTING,NOAV,NOEU,NOLV,NOP,word2vec,word2vec2,word2vec3,is_brain
0,1,0,0,0,2,0.00,0,0,0,2,...,5,1,2,0,0,14,0.766091,0.295311,-0.482078,False
1,1,0,1,0,2,0.00,0,0,0,2,...,6,1,3,0,0,13,0.350905,0.692511,0.782078,False
2,0,0,32,0,3,0.00,0,0,0,16,...,118,3,34,0,27,11,-0.064280,1.089711,2.046234,False
3,0,0,17,0,3,0.17,1,6,0,17,...,76,3,16,0,13,10,-0.479466,1.486911,3.310390,False
4,0,0,22,0,3,0.00,0,0,0,9,...,131,3,37,0,20,9,-0.894651,1.884111,4.574546,False


In [26]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [27]:
df = clean_dataset(df)

In [28]:
x, y = df.iloc[:, 1:-1], df.iloc[:, -1]
x.head()

,ATFD,CALIN,CC,CCL,CDISP,CEXT,CINT,CM,CYCLO,FANIN,...,LAA,LOC,MAXNESTING,NOAV,NOEU,NOLV,NOP,word2vec,word2vec2,word2vec3
0,0.0,0.0,0.0,2.0,0.00,0.0,0.0,0.0,2.0,0.0,...,1.0,5.0,1.0,2.0,0.0,0.0,14.0,0.766091,0.295311,-0.482078
1,0.0,1.0,0.0,2.0,0.00,0.0,0.0,0.0,2.0,0.0,...,1.0,6.0,1.0,3.0,0.0,0.0,13.0,0.350905,0.692511,0.782078
2,0.0,32.0,0.0,3.0,0.00,0.0,0.0,0.0,16.0,1.0,...,1.0,118.0,3.0,34.0,0.0,27.0,11.0,-0.064280,1.089711,2.046234
3,0.0,17.0,0.0,3.0,0.17,1.0,6.0,0.0,17.0,1.0,...,1.0,76.0,3.0,16.0,0.0,13.0,10.0,-0.479466,1.486911,3.310390
4,0.0,22.0,0.0,3.0,0.00,0.0,0.0,0.0,9.0,3.0,...,1.0,131.0,3.0,37.0,0.0,20.0,9.0,-0.894651,1.884111,4.574546


In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc  
from sklearn import model_selection
from sklearn.svm import SVC

In [29]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
scoring = {'accuracy' : make_scorer(accuracy_score),'precision' : make_scorer(precision_score),'recall' : make_scorer(recall_score),'f1_score' : make_scorer(f1_score)}
kfold = model_selection.KFold(n_splits=10, shuffle=True,random_state=42)

#svm

In [30]:
random_state = np.random.RandomState(0)
svm = SVC(kernel='linear', random_state=random_state)
ml = SVC(kernel='linear', probability=True,random_state=random_state,tol = 0.0001, max_iter = 100)

In [31]:
results = sklearn.model_selection.cross_validate(estimator=ml,X=x,y=y,cv=kfold,scoring=scoring)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-proce

In [32]:
results

{'fit_time': array([4.81608963, 5.33263516, 4.8642931 , 4.90532351, 6.19032979,
        6.92349267, 4.97075057, 4.70391297, 4.96255898, 5.06858468]),
 'score_time': array([0.04341865, 0.03568554, 0.03675961, 0.03812671, 0.08139825,
        0.03663802, 0.03658628, 0.0377121 , 0.03656244, 0.03889561]),
 'test_accuracy': array([0.16131532, 0.83702391, 0.83669176, 0.15422941, 0.84708227,
        0.19875983, 0.16066881, 0.16144392, 0.84519987, 0.27472041]),
 'test_precision': array([0.15614094, 0.44444444, 0.33146067, 0.15385469, 0.58518519,
        0.15613515, 0.15967975, 0.14895312, 0.58064516, 0.104     ]),
 'test_recall': array([0.94611187, 0.13483146, 0.04169611, 0.99213162, 0.11072179,
        0.91777003, 0.98423578, 0.93300071, 0.03821656, 0.48216833]),
 'test_f1_score': array([0.26804522, 0.20689655, 0.07407407, 0.26639777, 0.18621096,
        0.2668693 , 0.27477995, 0.25689334, 0.07171315, 0.17109593])}

In [33]:
accuracy = results['test_accuracy']
precision = results['test_precision']
recall = results['test_recall']
f1_score = results['test_f1_score']

In [34]:
A = np.mean(accuracy)
P = np.mean(precision)
R = np.mean(recall)
F1 = np.mean(f1_score)

In [35]:
print('Accuracy :' ,A ,'\n','precision : ',P,'\n','recall :  ',R,'\n','F1  : ',F1  ,'\n')

Accuracy : 0.4477135504328708 
 precision :  0.2820499115046938 
 recall :   0.5580884270345248 
 F1  :  0.20429762371776533 



#Naive Bayes

In [42]:
from sklearn.naive_bayes import GaussianNB
ml = GaussianNB()

In [43]:
results = sklearn.model_selection.cross_validate(estimator=ml,X=x,y=y,cv=kfold,scoring=scoring)

In [44]:
accuracy = results['test_accuracy']
precision = results['test_precision']
recall = results['test_recall']
f1_score = results['test_f1_score']

In [45]:
A = np.mean(accuracy)
P = np.mean(precision)
R = np.mean(recall)
F1 = np.mean(f1_score)

In [46]:
print('Accuracy :' ,A ,'\n','precision : ',P,'\n','recall :  ',R,'\n','F1  : ',F1  ,'\n')

Accuracy : 0.8711385085953814 
 precision :  0.6079926357056692 
 recall :   0.5151398400282327 
 F1  :  0.5576087263623856 

